# Productionize a model - TensoFlow Serving

 - This notebook is end-to-end example of training, evaluating and deploying an ML application using TensorFlow Serving.
 - High Level Estimators
 - Export the trained model for Deployment on TF serving
 - Although TF server is a Server, called by a client to make interfacing on new data, this notebook simulates client also
 
 

##### load dataset

In [35]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [36]:
print("TF verson is : ", tf.__version__)


TF verson is :  1.10.0


In [37]:
iris_training = "./data/iris_training.csv"
iris_test = "./data/iris_test.csv"

model_path = "./model"
model_path_serving = "./model_serving"

INPUT_TENSOR_NAME = "inputs"

##### input functions

In [38]:
pd.read_csv(iris_training).head()

,120,4,setosa,versicolor,virginica
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [44]:
def train_input_fn():
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=iris_training,
        target_dtype=np.int,
        features_dtype=np.float32
    )
    #above step is to make sure that our input X is a numpy float32 array
    return(tf.estimator.inputs.numpy_input_fn(x={INPUT_TENSOR_NAME:np.array(training_set.data)},
                                              y=np.array(training_set.target),
                                              num_epochs=None,
                                              shuffle=True
    )()
           
           
    )
def test_input_fn():
    testing_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=iris_test,
        target_dtype=np.int,
        features_dtype=np.float32
    )
    #above step is to make sure that our input X is a numpy float32 array
    return(tf.estimator.inputs.numpy_input_fn(x={INPUT_TENSOR_NAME:np.array(testing_set.data)},
                                              y=np.array(training_set.target),
                                              num_epochs=1,
                                              shuffle=False
    )()
    )



##### Define Estimator

In [45]:
def estimator(model_path):
    feature_columns = [tf.feature_column.numeric_column(INPUT_TENSOR_NAME, shape=[4])]
    return tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[10, 20, 30],
                                      n_classes=3,
                                      model_dir=model_path)

##### Training_stage

In [46]:
classifier = estimator(model_path)
classifier.train(input_fn=train_input_fn, steps=2000)
# accuraccy_score = classifier.evaluate(input_fn=test_input_fn)["accuraccy"]
# print("Accuraccy is : ", accuraccy_score)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000020A492CF7F0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving

##### Export the saved moder fto serve the client

- model is saved at  "./model"
- We need to create a service from the saved model to respond to requests
- next stpe  is to : export the model and run such a service locally and perform inference on new test data.

In [67]:
#How to feed inputs to TF server
def serving_input_receiver_fn():
    feature_spec={INPUT_TENSOR_NAME: tf.FixedLenFeature(dtype=tf.float32, shape=[4])}
    return(
        tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
    )

In [68]:
#Export the saved model 
save_path = classifier.export_savedmodel(export_dir_base= model_path_serving,
                                         serving_input_receiver_fn=serving_input_receiver_fn()
                                    )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from ./model\model.ckpt-4000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./model_serving\temp-b'1549957644'\saved_model.pb


##### Creating TF sessions with model path as an ID

In [106]:
sess_dict={}
def get_session(model_id):
    global sess_dict
    config = tf.ConfigProto(allow_soft_placement = True)
    sess_dict[model_id] = tf.Session(config = config)
    return(sess_dict[model_id])
def load_tf_model (model_path):
    sess = get_session(model_path)
    tf.saved_model.loader.load(sess, tags= [tf.saved_model.tag_constants.SERVING],export_dir= model_path)
    return sess

In [107]:
# Load the exported model and restore the session
sess = load_tf_model(save_path)

INFO:tensorflow:Restoring parameters from ./model_serving\1549957644\variables\variables


In [108]:
sess

#####  What is in the restored graph ??


* In real applications, the exported graphs would be provided as the models to be served
* We need to know what is the operation name of the graph : the type of operation the graph is provinding as service
* Solution : restored graph's get_operations() method

In [109]:
for op in tf.get_default_graph().get_operations():
    print(str(op.name))

    

global_step/Initializer/zeros
global_step
global_step/Assign
global_step/read
input_example_tensor
ParseExample/Const
ParseExample/ParseExample/names
ParseExample/ParseExample/dense_keys_0
ParseExample/ParseExample
dnn/input_from_feature_columns/input_layer/inputs/Shape
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack_1
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack_2
dnn/input_from_feature_columns/input_layer/inputs/strided_slice
dnn/input_from_feature_columns/input_layer/inputs/Reshape/shape/1
dnn/input_from_feature_columns/input_layer/inputs/Reshape/shape
dnn/input_from_feature_columns/input_layer/inputs/Reshape
dnn/input_from_feature_columns/input_layer/concat/concat_dim
dnn/input_from_feature_columns/input_layer/concat
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/shape
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/min
dnn/hiddenlayer_0/kernel

dnn/input_from_feature_columns/input_layer/inputs/Shape_8
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack_10
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack_1_8
dnn/input_from_feature_columns/input_layer/inputs/strided_slice/stack_2_8
dnn/input_from_feature_columns/input_layer/inputs/strided_slice_8
dnn/input_from_feature_columns/input_layer/inputs/Reshape/shape/1_8
dnn/input_from_feature_columns/input_layer/inputs/Reshape/shape_8
dnn/input_from_feature_columns/input_layer/inputs/Reshape_8
dnn/input_from_feature_columns/input_layer/concat/concat_dim_8
dnn/input_from_feature_columns/input_layer/concat_8
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/shape_8
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/min_8
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/max_8
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/RandomUniform_8
dnn/hiddenlayer_0/kernel/part_0/Initializer/random_uniform/sub_8
dnn/hidde

In [110]:
print(sess)

In [114]:
input_x_holder = sess.graph.get_operation_by_name("input_example_tensor").outputs[0]
print(input_x_holder)

Tensor("input_example_tensor:0", shape=(?,), dtype=string)


In [115]:
predictions_holder = sess.graph.get_operation_by_name("dnn/head/predictions/probabilities").outputs[0]
print(predictions_holder)

Tensor("dnn/head/predictions/probabilities:0", shape=(?, 3), dtype=float32)


##### How to use the Model as a server on a new data ??

In [118]:
# test set : [6.7, 2.5, 5.8, 1.8] , original label : class 2
float_features = tf.train.Feature(float_list = tf.train.FloatList(value = [6.7, 2.5, 5.8, 1.8]))
feature_dict = {"inputs" : float_features}

example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
serialized = example.SerializeToString()
score = sess.run([predictions_holder],{input_x_holder:[serialized]})

In [123]:
print(["%.4f" % score[0][0][idx] for idx in range(len(score[0][0]))])


['0.0000', '0.0052', '0.9948']


In [127]:
# Let us print the index of the largest score 
print("Predicted class is  : ",np.argmax(score))

Predicted class is  :  2
